In [ ]:
#He escogido Montreal como la ciudad donde, de cumplirse los requisitos, estableceré mi empresa.
#Buscaré un lugar en atención a los siguientes criterios: cercanía de colegios, cercanía de startups exitosas,
#lejanía de empresas con más de 10 años de antigüedad, cercanía de starbucks.
#Lo más importante, la panza del CEO: es vegano. Así que instalaremos nuestros headquarters indefectiblemente
#basándonos en la inmediata proximidad a un restaurante vegano, para que no pierda ni un minuto de su tiempo en
#desplazarse desde el comedor

In [5]:
from pymongo import MongoClient
import geocoder
import requests
import folium
import os
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [31]:
#usando un csv descargado de angel.co, voy a hallar las direcciones de algunas startups de Montreal
#potentes que hayan recaudado más de un millón de dólares

db, startups = connectCollection('companies','startups')

In [32]:
lista_startups=list(startups.find())

In [40]:
def loc_request(direction):
    if not os.getenv("GOOGLE"):
        raise ValueError("No token")
    else:
        g = geocoder.google(direction,key=os.getenv("GOOGLE"))
        return g.latlng

In [41]:
#prueba de mi función con el nombre de una startup
x=loc_request("breather montreal")
print(x)

[45.5281706, -73.59817389999999]


In [14]:
#saco las coordenadas de las empresas.
startup_loc = []
for startup in lista_startups:
    x = loc_request(startup["Name"]+"Montreal")
    startup_loc.append(x)

In [15]:
#limpio los errores
lista_def_startup_montreal=[startup for startup in startup_loc if startup!=None and str(startup[0])[0:2]=='45']

In [28]:
#Saco info sobre colegios en montreal. Me salen solo 20 resultados por búsqueda, de un máximo de 60 colegios
#que Google me devolverá.
#Usando el next_page_token, puedo repetir la búsqueda dos veces más y tener así 60 colegios de Montreal
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
params = {'key': os.getenv("GOOGLE"), 'location': '45.508888, -73.561668', 'radius': '15000', 'type': 'primary_school'}
r = requests.get(url, params=params)
results = r.json()['results']
token=r.json()['next_page_token']
token

'CrQCJQEAALWhsuAsgyqk7mfZLhVU-9VD7FLxFQioCKruXxfiqZKCiAvhFM66abUIjIPa6j2q05YEKUEzXU1D9zIzaCwFUYFxkSAptA49oIaBT8z1z-CzG3gKM8db4SmZfYf3RbK4iXcuxEWSJLMcxZBgqgUF0KDjhOnpOVfAOipdNWurtCK3lCCRmKc6JunKvEuwCwoyhonKzaYp4y9DIccJihAj08JJZ-0Q2HFXlm8SWV3d4pntBWKcB0sPg3xxF3t4EwRT0gCNFrUy538aUC2gpSo3kJ7PLF1XBlV5VARcu580PeY3Ohlg2JZOBFy5dDz5sDf-7ml7riLf-MLoIcONv2qj6hw04RlQvFCxX9GrgetzRX7QG_SLx1kn4fwnmUkPI5k-w2_fOEuFm1kO4ddo1DWTR7kSEEPjzlGO245kd8D4f3BK-OgaFLNhE3Ix82bOKUJhAQ7cMkXLXEVP'

In [30]:
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
params = {'key': os.getenv("GOOGLE"), 'location': '45.508888, -73.561668', 'radius': '15000', 'type': 'primary_school', "pagetoken": 'CrQCJQEAALWhsuAsgyqk7mfZLhVU-9VD7FLxFQioCKruXxfiqZKCiAvhFM66abUIjIPa6j2q05YEKUEzXU1D9zIzaCwFUYFxkSAptA49oIaBT8z1z-CzG3gKM8db4SmZfYf3RbK4iXcuxEWSJLMcxZBgqgUF0KDjhOnpOVfAOipdNWurtCK3lCCRmKc6JunKvEuwCwoyhonKzaYp4y9DIccJihAj08JJZ-0Q2HFXlm8SWV3d4pntBWKcB0sPg3xxF3t4EwRT0gCNFrUy538aUC2gpSo3kJ7PLF1XBlV5VARcu580PeY3Ohlg2JZOBFy5dDz5sDf-7ml7riLf-MLoIcONv2qj6hw04RlQvFCxX9GrgetzRX7QG_SLx1kn4fwnmUkPI5k-w2_fOEuFm1kO4ddo1DWTR7kSEEPjzlGO245kd8D4f3BK-OgaFLNhE3Ix82bOKUJhAQ7cMkXLXEVP'}
r = requests.get(url, params=params)
newresults = r.json()['results']
newtoken=r.json()['next_page_token']

In [32]:
print(newtoken)

CtQDxQEAAGkB6HUZsdOI2UG36kM9iC6rcZ4ybIVbhliFBQupeVQ6jpZKRKZRQPnkk8bn6VtDw5dOAPzBlKwGjwzVEHUCK9NxanInmk8gRJEJ0ZLOABgrcakkxO0qj99bWou4Mczn1wGepoDk-iIutuDizUq7Rt8ZX-SU6NObPmA6b3iG2CxMYBoqY_EgI9hso6VJrE-RQfHdYHyujkLEIW2uvIokiquu8d5FaLnGpACEnasFZ1GYwy0WWqjCPdawNZh4Fau795a2EAbkRBjyMS-ROLYyTPjffERhDC47p1VtYYzVB5qT5blA6LaAxcsJNkPD-naSxC8lynm176whXHu7eprDsc-iHunj2Z1QNXu9mzLtO4BdPu24BpRv5-QFjg-DTDkY23QRd2K-yakInySNqGSawr7TpVqfaBxYuYGfKsPt1LtA1SgSOd0LkLBjnm_SF-48pbgsti5NQGsTnoGauh3iU7D3eaktfhCY6BG_5jIDeIHOiSfuvLgRRMk0FP0FvQuj9CK7FbMbvTpOTHAMH9DRxw30wnMadkHPvE1HlH_1Z9RUEW3J4Gr0W2IWp0DHOekGMjayFrIuCp3lNdNrb_rjsqAVNxlkY0GnhsZZntXoahfpEhAypF1-vmr3SF1u9rN4EJGgGhS7s0eqw7eDxnwtGIXG71k_DTFXAQ


In [33]:
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
params = {'key': os.getenv("GOOGLE"), 'location': '45.508888, -73.561668', 'radius': '15000', 'type': 'primary_school', "pagetoken":"CtQDxQEAAGkB6HUZsdOI2UG36kM9iC6rcZ4ybIVbhliFBQupeVQ6jpZKRKZRQPnkk8bn6VtDw5dOAPzBlKwGjwzVEHUCK9NxanInmk8gRJEJ0ZLOABgrcakkxO0qj99bWou4Mczn1wGepoDk-iIutuDizUq7Rt8ZX-SU6NObPmA6b3iG2CxMYBoqY_EgI9hso6VJrE-RQfHdYHyujkLEIW2uvIokiquu8d5FaLnGpACEnasFZ1GYwy0WWqjCPdawNZh4Fau795a2EAbkRBjyMS-ROLYyTPjffERhDC47p1VtYYzVB5qT5blA6LaAxcsJNkPD-naSxC8lynm176whXHu7eprDsc-iHunj2Z1QNXu9mzLtO4BdPu24BpRv5-QFjg-DTDkY23QRd2K-yakInySNqGSawr7TpVqfaBxYuYGfKsPt1LtA1SgSOd0LkLBjnm_SF-48pbgsti5NQGsTnoGauh3iU7D3eaktfhCY6BG_5jIDeIHOiSfuvLgRRMk0FP0FvQuj9CK7FbMbvTpOTHAMH9DRxw30wnMadkHPvE1HlH_1Z9RUEW3J4Gr0W2IWp0DHOekGMjayFrIuCp3lNdNrb_rjsqAVNxlkY0GnhsZZntXoahfpEhAypF1-vmr3SF1u9rN4EJGgGhS7s0eqw7eDxnwtGIXG71k_DTFXAQ"}
r = requests.get(url, params=params)
new2results = r.json()['results']

In [36]:
lista_schools=[]
for school in range(0,len(results)):
    location = results[school]['geometry']['location']
    listilla=[location['lat'], location['lng']]
    lista_schools.append(listilla)
for school in range(0,len(newresults)):
    location = newresults[school]['geometry']['location']
    listilla=[location['lat'], location['lng']]
    lista_schools.append(listilla)
for school in range(0,len(new2results)):
    location = new2results[school]['geometry']['location']
    listilla=[location['lat'], location['lng']]
    lista_schools.append(listilla)

In [49]:
airport_coord=loc_request("airport montreal")

In [17]:
db, companies = connectCollection('companies','companies')

In [18]:
#saco las compañías de montreal que tienen más de 10 años: no nos interesa situarnos cerca de semejantes vejestorios
compan = list(companies.find({"offices.city": "Montreal","founded_year": {"$lt": 2008},"offices.longitude":{"$ne":None},"offices.latitude":{"$ne": None}}))

In [56]:
#filtrado de las coordenadas de las compañías viejas
bad_companies=[]

for c in compan:
    for i in range(0,len(c['offices'])):
        coord=[]
        coord.append(c['offices'][i]['latitude'])
        coord.append(c['offices'][i]['longitude'])
        bad_companies.append(coord)
    
old_companies=[m for m in bad_companies if str(m[0])[:2]=="45"]
print(old_companies)

[[45.506643, -73.566634], [45.500277, -73.560084], [45.545447, -73.639076], [45.516545, -73.578477], [45.5075577, -73.5696547], [45.545447, -73.639076], [45.539065, -73.609532], [45.499215, -73.559631], [45.496397, -73.5721802], [45.498179, -73.565316], [45.545447, -73.639076], [45.50306, -73.556133], [45.496141, -73.63222], [45.496141, -73.63222], [45.4938086, -73.5749531], [45.5130285, -73.5707374], [45.4874956, -73.5909495], [45.4961978, -73.6763827], [45.545447, -73.639076]]


In [ ]:
#aquí detecto un error: quizás algunas de las startups exitosas que saqué de angel.co tengan ya más de 10 años. Habremos de filtrarlas
#para no representarlas en el mapa como startups

In [37]:
nombres_startups=[st['Name'].strip() for st in lista_startups]
nombres_viejunas=[v['name'].strip() for v in compan]

In [43]:
coord_fuera_de_startups=[]
for v in nombres_startups:
    if v in nombres_viejunas:
        x = loc_request(v +"Montreal")
        coord_fuera_de_startups.append(x)
coord_fuera_de_startups

[[45.507406, -73.5701019],
 [45.5016889, -73.567256],
 [45.5049622, -73.5534637],
 [45.5016889, -73.567256],
 [45.5016889, -73.567256]]

In [54]:
#saco la lista definitiva de coordenadas de las startups potentes, para poder pintarla
startups_mtl=[]
for st in lista_def_startup_montreal:
    if st not in coord_fuera_de_startups:
        startups_mtl.append(st)
print(startups_mtl)

[[45.5281706, -73.59817389999999], [45.526562, -73.595941], [45.5990964, -73.60740779999999], [45.4940124, -73.6577229], [45.4994997, -73.56582279999999], [45.4974239, -73.5545678], [45.5026306, -73.5735681], [45.50294, -73.5696979], [45.5283051, -73.5980458], [45.49761609999999, -73.5587199], [45.4971731, -73.5681743], [45.5018405, -73.556189], [45.51219740000001, -73.5532286], [45.51800100000001, -73.58219], [45.526123, -73.5950714], [45.527967, -73.5975509], [45.4974749, -73.5687438], [45.4750676, -73.5803263], [45.484051, -73.562141], [45.49754799999999, -73.6592847], [45.5194347, -73.58459239999999], [45.519511, -73.58732309999999], [45.5316578, -73.622681], [45.527514, -73.596334], [45.4629274, -73.5952171], [45.5005628, -73.5744579], [45.52824630000001, -73.5980107], [45.51800069999999, -73.58218959999999], [45.5017173, -73.5588095], [45.5002463, -73.5604499], [45.48450589999999, -73.5625362], [45.5024127, -73.5631131], [45.5178237, -73.58216089999999], [45.4826805, -73.5829534]

In [58]:
#guardo aquí para futuros reinicios de Kernel las listas que deseo pintar en el mapa
startups_mtl=[[45.5281706, -73.59817389999999], [45.526562, -73.595941], [45.5990964, -73.60740779999999], [45.4940124, -73.6577229], [45.4994997, -73.56582279999999], [45.4974239, -73.5545678], [45.5026306, -73.5735681], [45.50294, -73.5696979], [45.5283051, -73.5980458], [45.49761609999999, -73.5587199], [45.4971731, -73.5681743], [45.5018405, -73.556189], [45.51219740000001, -73.5532286], [45.51800100000001, -73.58219], [45.526123, -73.5950714], [45.527967, -73.5975509], [45.4974749, -73.5687438], [45.4750676, -73.5803263], [45.484051, -73.562141], [45.49754799999999, -73.6592847], [45.5194347, -73.58459239999999], [45.519511, -73.58732309999999], [45.5316578, -73.622681], [45.527514, -73.596334], [45.4629274, -73.5952171], [45.5005628, -73.5744579], [45.52824630000001, -73.5980107], [45.51800069999999, -73.58218959999999], [45.5017173, -73.5588095], [45.5002463, -73.5604499], [45.48450589999999, -73.5625362], [45.5024127, -73.5631131], [45.5178237, -73.58216089999999], [45.4826805, -73.5829534], [45.5022594, -73.5557439], [45.4730348, -73.5886763], [45.5015765, -73.5755787], [45.5104355, -73.5543955], [45.497274, -73.5545995], [45.4725006, -73.590389], [45.5019854, -73.5721975], [45.4583646, -73.8351037], [45.4962148, -73.6178254], [45.51006, -73.568715], [45.491499, -73.602397], [45.48011330000001, -73.76254279999999], [45.5195964, -73.6147503], [45.5193495, -73.58721129999999], [45.4827825, -73.76598589999999], [45.4993916, -73.5601009], [45.50443509999999, -73.5681413], [45.4971041, -73.554168], [45.5051223, -73.56769059999999]]
old_companies=[[45.506643, -73.566634], [45.500277, -73.560084], [45.545447, -73.639076], [45.516545, -73.578477], [45.5075577, -73.5696547], [45.545447, -73.639076], [45.539065, -73.609532], [45.499215, -73.559631], [45.496397, -73.5721802], [45.498179, -73.565316], [45.545447, -73.639076], [45.50306, -73.556133], [45.496141, -73.63222], [45.496141, -73.63222], [45.4938086, -73.5749531], [45.5130285, -73.5707374], [45.4874956, -73.5909495], [45.4961978, -73.6763827], [45.545447, -73.639076]]
airport_coord=[45.46568509999999, -73.7454814]
lista_schools=[[45.4515859, -73.5973666], [45.464211, -73.58437599999999], [45.4907729, -73.600489], [45.48093739999999, -73.5949428], [45.4543059, -73.64229329999999], [45.4983836, -73.58638289999999], [45.5495294, -73.6240947], [45.5195553, -73.5884734], [45.5400483, -73.4576554], [45.5468238, -73.6020272], [45.52529879999999, -73.56732869999999], [45.4322217, -73.6237926], [45.6158818, -73.4527145], [45.467306, -73.634286], [45.5568909, -73.6001124], [45.5265098, -73.6802211], [45.5052242, -73.5153851], [45.4906038, -73.46610609999999], [45.5006965, -73.6892749], [45.49372, -73.63751680000001], [45.495706, -73.466617], [45.5247229, -73.5575313], [45.4809967, -73.5616277], [45.5221578, -73.5966231], [45.478643, -73.5860612], [45.5087145, -73.6195948], [45.5088101, -73.6199002], [45.4712164, -73.58925640000001], [45.51750089999999, -73.50009899999999], [45.51786719999999, -73.49785949999999], [45.5550324, -73.57399649999999], [45.4816061, -73.6167957], [45.54721379999999, -73.611826], [45.4561235, -73.57174789999999], [45.45641570000001, -73.5771714], [45.5402461, -73.62944379999999], [45.47355200000001, -73.628762], [45.5303601, -73.6416009], [45.52426730000001, -73.4767986], [45.5201898, -73.6904689], [45.4472914, -73.57818499999999], [45.4485184, -73.5964904], [45.539655, -73.6471348], [45.4823806, -73.4713347], [45.541771, -73.474914], [45.54998629999999, -73.651763], [45.4343613, -73.59285100000001], [45.5781286, -73.6137425], [45.57080099999999, -73.63136399999999], [45.579179, -73.61345610000001], [45.48067569999999, -73.4507371], [45.5608367, -73.6542271], [45.51058889999999, -73.68244820000001], [45.58338699999999, -73.6225104], [45.46323899999999, -73.6656536], [45.5501522, -73.67036709999999], [45.50667989999999, -73.68768840000001], [45.4806866, -73.4380252], [45.45927570000001, -73.6781438], [45.4417141, -73.6735605]]

In [61]:
#mapa hasta el momento
mtl_map=folium.Map(location=[45.508888, -73.561668], zoom_start=11)
for startup in startups_mtl:
    folium.Marker(startup,radius=2, icon=folium.Icon(icon='usd',color='lightblue')).add_to(mtl_map)
for school in lista_schools:
    folium.Marker(school,radius=2, icon=folium.Icon(icon='home',color='orange')).add_to(mtl_map)
folium.Marker(airport_coord,radius=2, icon=folium.Icon(icon='plane',color='green')).add_to(mtl_map)
for old in old_companies:
    folium.Marker(old,radius=2, icon=folium.Icon(icon='usd',color='red')).add_to(mtl_map)

In [62]:
mtl_map

In [63]:
#starbucks
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
params = {'key': os.getenv("GOOGLE"), 'location': '45.508888, -73.561668', 'radius': '15000', 'name': "starbucks"}
r = requests.get(url, params=params)
results = r.json()['results']
token=r.json()['next_page_token']
token

'CqQCIAEAANvWttSM9TF94mysIzFKO-0hSf_Hsep7fyUiULKFCUufWH19mLfGWF_z5Ha6UM2IZuRggFuXYl7qwLKTcPYKMs_qIP-5gpw5fFcNGFBS4D36qiLFoPFcTIKoHk6u1Eu_XX7DtfU37QEERR9tYmErRDDj2Xs-aahVp9SjfWRKCpC6-3-Ard1B-tXs0R7FS-z6nRFifqbvKveWgNkWZhaZG4mS2YgKmDLDCNGeJpQ0w1VtlvSKtwY66zZYChS09s68Z1ingB-xPsRYJ9THyj077aAOh0fQwCfnULgjVMJ6SQ5H31yKECq5VYhRd9lSCzBEbRp3rjAbim4q-zBiKJEx7ncBydSKuDlEGfLdfkCkMj_XlTHov6l9j84ysrXFcL5nXxIQOiWuxrmnPBQUXBvKFCo2LRoUEapPN8xEI9usa-emCAo1lYGZlFM'

In [66]:
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
params = {'key': os.getenv("GOOGLE"), 'location': '45.508888, -73.561668', 'radius': '15000', 'name': "starbucks", "pagetoken":"CqQCIAEAANvWttSM9TF94mysIzFKO-0hSf_Hsep7fyUiULKFCUufWH19mLfGWF_z5Ha6UM2IZuRggFuXYl7qwLKTcPYKMs_qIP-5gpw5fFcNGFBS4D36qiLFoPFcTIKoHk6u1Eu_XX7DtfU37QEERR9tYmErRDDj2Xs-aahVp9SjfWRKCpC6-3-Ard1B-tXs0R7FS-z6nRFifqbvKveWgNkWZhaZG4mS2YgKmDLDCNGeJpQ0w1VtlvSKtwY66zZYChS09s68Z1ingB-xPsRYJ9THyj077aAOh0fQwCfnULgjVMJ6SQ5H31yKECq5VYhRd9lSCzBEbRp3rjAbim4q-zBiKJEx7ncBydSKuDlEGfLdfkCkMj_XlTHov6l9j84ysrXFcL5nXxIQOiWuxrmnPBQUXBvKFCo2LRoUEapPN8xEI9usa-emCAo1lYGZlFM"}
newr = requests.get(url, params=params)
newresults = newr.json()['results']
newtoken=newr.json()['next_page_token']
newtoken

'CsQDwAEAAK-6SWOQ-TmgGVhZODtbQ4YupM5miLSAAqav8iyjAl9Au2r0HyqyOfN5046n3m8Lg652WcEnl_G1trZYHGIKomQWEOWcsOKIsoIyIGjRpJJ1eGvT-SU5gm0Il9Jw2DlMBIej3bAz8kQersVBQpskcvAG9xNZID7iKhaUS_L1xYd9Jjgcu10Yg9x6UsCmTk-5lJoYGugXHhymNEIFJ4c7WOhuzCoUlUtiE3rfkojGatpIhmqXeXSuyJkLXnP_mLFj8AHo7OLVWSjAMa_Ko7gbA9Iw_lVAhWh-VCS_zA29yqZ2tMDOvWZSzxn0MqHM3Qn_bqM1aKlYqqODK69JWCwpzPtRGqhKvTJGvjl6YPM7iyA-dHT8K8c4rBtiIAuj9OfrMJx135IZyVMF8VRFfukZal0U7EdGWZ7I-D42OMvHBInt0aXUePr-xtrhZ_TO-_TI4YbfYk012vAbW0Fw6xZIQdrPepj-xuqtUPM1ylbkOtg05YQ2RQURlchEh_7t9dWdXhBJgq_ZyfTIQYLgtgREAZgFF9BJWoEnuBrrnAubREevvdSuZJkxZuOPYN6qavJebidtb2NJC41kAacbyE4HM18SEBDBQKRIXbwN9g5NamVklZkaFGAECrKFzZA0KZ66pfoCg1Vsafx7'

In [68]:
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
params = {'key': os.getenv("GOOGLE"), 'location': '45.508888, -73.561668', 'radius': '15000', 'name': "starbucks", "pagetoken":"CqQCIAEAANvWttSM9TF94mysIzFKO-0hSf_Hsep7fyUiULKFCUufWH19mLfGWF_z5Ha6UM2IZuRggFuXYl7qwLKTcPYKMs_qIP-5gpw5fFcNGFBS4D36qiLFoPFcTIKoHk6u1Eu_XX7DtfU37QEERR9tYmErRDDj2Xs-aahVp9SjfWRKCpC6-3-Ard1B-tXs0R7FS-z6nRFifqbvKveWgNkWZhaZG4mS2YgKmDLDCNGeJpQ0w1VtlvSKtwY66zZYChS09s68Z1ingB-xPsRYJ9THyj077aAOh0fQwCfnULgjVMJ6SQ5H31yKECq5VYhRd9lSCzBEbRp3rjAbim4q-zBiKJEx7ncBydSKuDlEGfLdfkCkMj_XlTHov6l9j84ysrXFcL5nXxIQOiWuxrmnPBQUXBvKFCo2LRoUEapPN8xEI9usa-emCAo1lYGZlFM"}
newr2 = requests.get(url, params=params)
newresults2 = newr2.json()['results']